# Import libraries

In [1]:
from gumps.interface import model_interface
from gumps.interface import model_units

import attrs
import pandas as pd

# Create Units

In [2]:
@attrs.define
class AllowableSimulationTime(model_units.Time):
    @classmethod
    def bounds_units(cls):
        return 'hr'

    @classmethod
    def lower_bound(cls) -> float|int:
        return 0.0

    @classmethod
    def upper_bound(cls) -> float|int:
        return 24.0


@attrs.define
class ReactorVolume(model_units.Volume):
    @classmethod
    def bounds_units(cls):
        return 'liters'

    @classmethod
    def lower_bound(cls) -> float|int:
        return 10.0

    @classmethod
    def upper_bound(cls) -> float|int:
        return 100.0
    

@attrs.define
class ProductDensity(model_units.Density):
    @classmethod
    def bounds_units(cls):
        return 'kg/m^3'

    @classmethod
    def lower_bound(cls) -> float|int:
        return 1050.0

    @classmethod
    def upper_bound(cls) -> float|int:
        return 1150.0
    

@attrs.define
class ProductViscosity(model_units.Viscosity):
    @classmethod
    def bounds_units(cls):
        return 'cP'

    @classmethod
    def lower_bound(cls) -> float|int:
        return 0.0

    @classmethod
    def upper_bound(cls) -> float|int:
        return 100.0
    
@attrs.define
class MediaDensity(model_units.Density):
    @classmethod
    def bounds_units(cls):
        return 'kg/m^3'

    @classmethod
    def lower_bound(cls) -> float|int:
        return 950.0

    @classmethod
    def upper_bound(cls) -> float|int:
        return 1050.0

# Create Units for Pandas Dataframes and Series

In [3]:
@attrs.define
class ProteinInformation(model_interface.Units):
    @staticmethod
    def units_used() -> dict[str, model_interface.Unit]:
        return {'product_density': ProductDensity,
                'product_viscosity': ProductViscosity,
                'media_density': MediaDensity}


# Create a collection of Units

In [4]:
@attrs.define
class Reactor(model_interface.UnitCollection):
    simulation_time: AllowableSimulationTime
    reactor_volume: ReactorVolume
    protein: ProteinInformation

# Work with the units

In [5]:
# Lets make a basic unit and then convert it

sim_time = AllowableSimulationTime(0.5, 'day')

sim_time.convert('s')

AllowableSimulationTime(value=43200.0, units=<Quantity(1, 'second')>)

In [6]:
#lets make some proteins

df = pd.DataFrame({'product_density': [1100.0, 1050.0, 1140.0],
                   'product_viscosity': [1.0, 1.1, 1.2],
                   'media_density': [950.0, 990.0, 1010.0]})

units = {'product_density': 'kg/m^3',
         'product_viscosity': 'cP',
         'media_density': 'kg/m^3'}

protein1 = ProteinInformation(df, units)

print(protein1)

ProteinInformation(value=   product_density  product_viscosity  media_density
0           1100.0                1.0          950.0
1           1050.0                1.1          990.0
2           1140.0                1.2         1010.0, units={'product_density': <Quantity(1.0, 'kilogram / meter ** 3')>, 'product_viscosity': <Quantity(1, 'centipoise')>, 'media_density': <Quantity(1.0, 'kilogram / meter ** 3')>})


In [7]:
#Create a reactor

reactor1 = Reactor(AllowableSimulationTime(5.0, 'hr'),
                     ReactorVolume(50.0, 'liters'),
                     protein1[0])

In [8]:
protein1.convert({ProductDensity: 'g/mL',
                  ProductViscosity: 'Pa*s',
                  MediaDensity: 'g/L'})

ProteinInformation(value=   product_density  product_viscosity  media_density
0             1.10             0.0010          950.0
1             1.05             0.0011          990.0
2             1.14             0.0012         1010.0, units={'product_density': <Quantity(1.0, 'gram / milliliter')>, 'product_viscosity': <Quantity(1, 'pascal * second')>, 'media_density': <Quantity(1.0, 'gram / liter')>})

In [9]:
reactor1.convert({AllowableSimulationTime: 's',
                  ReactorVolume: 'm^3'})

Reactor(simulation_time=AllowableSimulationTime(value=18000.0, units=<Quantity(1, 'second')>), reactor_volume=ReactorVolume(value=0.05000000000000001, units=<Quantity(1, 'meter ** 3')>), protein=ProteinInformation(value=product_density      1100.0
product_viscosity       1.0
media_density         950.0
dtype: float64, units={'product_density': <Quantity(1.0, 'kilogram / meter ** 3')>, 'product_viscosity': <Quantity(1, 'centipoise')>, 'media_density': <Quantity(1.0, 'kilogram / meter ** 3')>}))